<a href="https://colab.research.google.com/github/sergiolitwiniuk/anyoneAI/blob/main/Sprint1_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import time

In [ ]:
!pip install nba_api

In [ ]:
from nba_api.stats.endpoints import playercareerstats, commonplayerinfo
from nba_api.stats.static import players

In [ ]:
nba_players = players.get_players()  #API call to pull players data

In [ ]:
print('Number of players fetched: {}'.format(len(nba_players)))
nba_players[:1]

Number of players fetched: 4723


[{'first_name': 'Alaa',
  'full_name': 'Alaa Abdelnaby',
  'id': 76001,
  'is_active': False,
  'last_name': 'Abdelnaby'}]

In [ ]:
players = pd.DataFrame.from_dict(nba_players)

In [ ]:
players = players.loc[players.is_active==True]   #Retrieve only active players f
players.sort_values('id')

,id,full_name,first_name,last_name,is_active
2033,2544,LeBron James,LeBron,James,True
114,2546,Carmelo Anthony,Carmelo,Anthony,True
1742,2617,Udonis Haslem,Udonis,Haslem,True
1916,2730,Dwight Howard,Dwight,Howard,True
1971,2738,Andre Iguodala,Andre,Iguodala,True
...,...,...,...,...,...
1504,1630692,Jordan Goodwin,Jordan,Goodwin,True
1150,1630693,Jaime Echenique,Jaime,Echenique,True
3379,1630695,Micah Potter,Micah,Potter,True
3894,1630696,Dru Smith,Dru,Smith,True


In [ ]:
career = playercareerstats.PlayerCareerStats(player_id = 203076)
commonCareer = career.get_data_frames()[0][career.get_data_frames()[0]['SEASON_ID'] == '2021-22']
commonCareer

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
9,203076,2021-22,00,1610612747,LAL,29.0,40,40,1404.0,370,...,0.713,106,288,394,122,49,90,82,97,927


In [ ]:
commonInfo = commonplayerinfo.CommonPlayerInfo(player_id = 203076,timeout=30)
commonInfo.get_data_frames()[0].loc[(commonInfo.get_data_frames()[0]['GAMES_PLAYED_FLAG'] != 'N') & (commonInfo.get_data_frames()[0]['PERSON_ID'] != 1630597)  & (commonInfo.get_data_frames()[0]['TEAM_ID'] != 0) ]


,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG
0,203076,Anthony,Davis,Anthony Davis,"Davis, Anthony",A. Davis,anthony-davis,1993-03-11T00:00:00,Kentucky,USA,...,anthony_davis,2012,2021,N,Y,Y,2012,1,1,Y


In [ ]:
commonPInfo = commonInfo.get_data_frames()[0]
commonFilteredInfo = commonPInfo.filter(['DISPLAY_FIRST_LAST','TEAM_NAME','POSITION','HEIGHT','WEIGHT','COUNTRY','BIRTHDATE','SEASON_EXP','DRAFT_NUMBER','GP','MIN','PTS','REB','AST','STL','BLK','GAME_DATE','AGE'], axis=1)
careerFilteredInfo = commonCareer['GP']

In [ ]:
 commonFilteredInfo
careerFilteredInfo
partialData = [commonFilteredInfo, careerFilteredInfo]
result = pd.concat(partialData)
pd.DataFrame(result)
result

,0,BIRTHDATE,COUNTRY,DISPLAY_FIRST_LAST,DRAFT_NUMBER,HEIGHT,POSITION,SEASON_EXP,TEAM_NAME,WEIGHT
0,NaN,1993-03-11T00:00:00,USA,Anthony Davis,1,6-10,Forward-Center,9.0,Lakers,253
9,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pl = players[:10]["id"]
pl

10    1630173
22     203500
24    1628389
40    1630583
43     200746
53    1629638
57    1628960
58    1628386
73    1630247
75    1630631
Name: id, dtype: int64

In [ ]:
prueba = []
for ele in players["id"]:
  career = playercareerstats.PlayerCareerStats(player_id = ele)
  prueba.append(career)
  print(len(prueba))
  time.sleep(0.5)
prueba

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
len(prueba)

587

In [ ]:
data_players = prueba[0].get_data_frames()[0].loc[prueba[2].get_data_frames()[0].SEASON_ID == "2021-22"]

In [ ]:
data_players = prueba[0].get_data_frames()[0].loc[prueba[2].get_data_frames()[0].SEASON_ID == "2021-22"]
for player in range(0,len(prueba)):
  data_players = pd.concat([data_players,prueba[player].get_data_frames()[0].loc[prueba[player].get_data_frames()[0].SEASON_ID == "2021-22"]], ignore_index=True)

In [ ]:
data_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707 entries, 0 to 706
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PLAYER_ID          707 non-null    object 
 1   SEASON_ID          707 non-null    object 
 2   LEAGUE_ID          707 non-null    object 
 3   TEAM_ID            707 non-null    object 
 4   TEAM_ABBREVIATION  707 non-null    object 
 5   PLAYER_AGE         707 non-null    float64
 6   GP                 707 non-null    object 
 7   GS                 707 non-null    object 
 8   MIN                707 non-null    float64
 9   FGM                707 non-null    object 
 10  FGA                707 non-null    object 
 11  FG_PCT             707 non-null    float64
 12  FG3M               707 non-null    object 
 13  FG3A               707 non-null    object 
 14  FG3_PCT            707 non-null    float64
 15  FTM                707 non-null    object 
 16  FTA                707 non

In [ ]:
data_players.drop(data_players.loc[data_players.TEAM_ID == 0].index , axis=0, inplace = True)

In [ ]:
data_players.head(15)

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1630173,2021-22,00,1610612761,TOR,22.0,73,28,1725.0,265,...,0.595,146,327,473,82,37,41,84,151,664
1,203500,2021-22,00,1610612763,MEM,28.0,76,75,1999.0,210,...,0.543,349,411,760,256,65,60,115,153,528
2,1628389,2021-22,00,1610612748,MIA,24.0,56,56,1825.0,406,...,0.753,137,427,564,190,80,44,148,171,1068
3,1630583,2021-22,00,1610612763,MEM,21.0,32,0,360.0,53,...,0.625,33,54,87,21,6,10,16,36,132
4,200746,2021-22,00,1610612751,BKN,36.0,47,12,1050.0,252,...,0.873,73,185,258,42,14,47,44,78,607
5,1629638,2021-22,00,1610612740,NOP,23.0,50,19,1317.0,237,...,0.722,36,128,164,139,41,19,85,88,639
6,1629638,2021-22,00,1610612762,UTA,23.0,15,2,149.0,16,...,0.917,1,22,23,17,5,4,8,15,53
8,1628960,2021-22,00,1610612749,MIL,26.0,66,61,1805.0,255,...,0.865,32,190,222,100,46,18,43,96,733
9,1628386,2021-22,00,1610612739,CLE,23.0,56,56,1810.0,369,...,0.708,192,410,602,92,44,75,94,97,904
10,1630631,2021-22,00,1610612740,NOP,23.0,54,1,834.0,131,...,0.679,25,75,100,152,71,7,40,73,330
